LIBRARY

In [1]:
import dill  # in order to save Lambda Layer
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from resnet import *

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


# Loading the Model

In [2]:
model = resnet20()

check_point = torch.load('D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-10-23_Pytorch hook/pretrained_models/resnet20-12fca82f.th', map_location=device)

model = torch.nn.DataParallel(model)
model.load_state_dict(check_point['state_dict'])

torch.save(model.module, 'D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-10-23_Pytorch hook/save_temp/resnet20_check_point.pth', pickle_module=dill)

net = torch.load('D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-10-23_Pytorch hook/save_temp/resnet20_check_point.pth', map_location=device)

net.eval()

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=

# LOAD DATASET

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                     
dataset = torch.utils.data.DataLoader(
    datasets.CIFAR10(root='./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])), batch_size=16, shuffle=False, num_workers=4, pin_memory=True)   

# Traverse all Resnet modules using hooks

In [4]:
net.dict={} # {module_id : module name}
net.hooks={} # hooks to remove later
net.name_ids=[] # module names
net.modules=[] # list of module objects
net.mdict={} # modules dict

def has_children(module):
    '''check if module has children'''
    if len(list(module.children())) > 0:
        return True
    else:
        return False

def hook(module, inp, outp): 
    if module.name:
        net.name_ids.append((module.name) + " " + module.__class__.__name__)
        net.modules.append(module)
    
for name, module in net.named_modules():           
    module.name = name
    module.type = module.__class__.__name__
    module.has_children = has_children(module)
    net.hooks[name]= module.register_forward_hook(hook)
    
with torch.no_grad():
    for images, labels in dataset:
        images, labels = images.cuda(), labels.cuda()
        out = net(images)

net.dict = dict(zip(range(len(net.name_ids)),net.name_ids))
net.mdict = dict(zip(range(len(net.modules)),net.modules))

In [5]:
net.dict

{0: 'conv1 Conv2d',
 1: 'bn1 BatchNorm2d',
 2: 'layer1.0.conv1 Conv2d',
 3: 'layer1.0.bn1 BatchNorm2d',
 4: 'layer1.0.conv2 Conv2d',
 5: 'layer1.0.bn2 BatchNorm2d',
 6: 'layer1.0.shortcut Sequential',
 7: 'layer1.0 BasicBlock',
 8: 'layer1.1.conv1 Conv2d',
 9: 'layer1.1.bn1 BatchNorm2d',
 10: 'layer1.1.conv2 Conv2d',
 11: 'layer1.1.bn2 BatchNorm2d',
 12: 'layer1.1.shortcut Sequential',
 13: 'layer1.1 BasicBlock',
 14: 'layer1.2.conv1 Conv2d',
 15: 'layer1.2.bn1 BatchNorm2d',
 16: 'layer1.2.conv2 Conv2d',
 17: 'layer1.2.bn2 BatchNorm2d',
 18: 'layer1.2.shortcut Sequential',
 19: 'layer1.2 BasicBlock',
 20: 'layer1 Sequential',
 21: 'layer2.0.conv1 Conv2d',
 22: 'layer2.0.bn1 BatchNorm2d',
 23: 'layer2.0.conv2 Conv2d',
 24: 'layer2.0.bn2 BatchNorm2d',
 25: 'layer2.0.shortcut LambdaLayer',
 26: 'layer2.0 BasicBlock',
 27: 'layer2.1.conv1 Conv2d',
 28: 'layer2.1.bn1 BatchNorm2d',
 29: 'layer2.1.conv2 Conv2d',
 30: 'layer2.1.bn2 BatchNorm2d',
 31: 'layer2.1.shortcut Sequential',
 32: 'layer

In [6]:
net.mdict

{0: Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 1: BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 2: Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 3: BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 4: Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
 5: BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 6: Sequential(),
 7: BasicBlock(
   (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
   (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (shortcut): Sequential()
 ),
 8: Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), b

# Prints the shape of each layer's output using hooks

In [7]:
def print_shape(module, input, output):
    print(f"{module} input shape: {input[0].shape}, output shape: {output.shape}")

# Attach the hook to all layers
handle = []
for name, module in net.named_modules():
    if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
        handle.append(module.register_forward_hook(print_shape))

with torch.no_grad():
    for images, labels in dataset:
        images, labels = images.cuda(), labels.cuda()
        out = net(images)

# Remove the hook
for h in handle:
    h.remove()

Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 3, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 16, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 16, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 16, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 16, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) input shape: torch.Size([16, 16, 32, 32]), output shape: torch.Size([16, 16, 32, 32])
Conv2d(16, 16, ker